# Import Modules

In [7]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from nanrms import nansumsq, nanrms

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

# Collect Data Files

In [8]:
datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
exp = "_Bandlock_Udp_B1_B3_B7_B8_RM500Q"
devices = [
    'qc00',
    # 'qc01',
    'qc02',
    'qc03',
]
schemes = [
    'B1',
    # 'B3',
    'B7',
    'B8',
]
dates = {
    "2023-03-15": [*['#{:02d}'.format(i+1) for i in range(1, 4)], '#06'],
    "2023-03-16": ['#{:02d}'.format(i+1) for i in range(16)],
}

In [9]:
def set_data(df):
    df['seq'] = df['seq'].astype('Int32')
    df['rpkg'] = df['rpkg'].astype('Int8')
    df['frame_id'] = df['frame_id'].astype('Int32')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['xmit_time'] = pd.to_datetime(df['xmit_time'])
    df['arr_time'] = pd.to_datetime(df['arr_time'])
    df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
    df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
    df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
    df['lost'] = df['lost'].astype('boolean')
    df['excl'] = df['excl'].astype('boolean')
    df['latency'] = df['latency'].astype('float32')
    return df

## Uplink

In [10]:
i = -1
dfs_ul = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        if i in [16,17,18,19]:
            dfs_ul.append([pd.DataFrame()]*len(devices))
            continue
        dfs_ul.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            # print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_ul[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ul))

20


## Downlink

In [11]:
i = -1
dfs_dl = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        if i in [0,1,2,3]:
            dfs_dl.append([pd.DataFrame()]*len(devices))
            continue
        dfs_dl.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            # print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_dl[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_dl))

20


In [12]:
df = dfs_ul[0][0]

plr = round(df.lost.mean() * 100, 4)
elr = round(df[~df.lost].excl.mean() * 100, 4)
plelr = round(df.excl.mean() * 100, 4)
avg_l = round(df[~df.lost].latency.mean(), 6)
min_l = round(df[~df.lost].latency.min(), 6)
max_l = round(df[~df.lost].latency.max(), 6)
mdn_l = round(df[~df.lost].latency.median(), 6)
var_l = round(df[~df.lost].latency.var(), 6)
std_l = round(df[~df.lost].latency.std(), 6)
jitter = round(df.loc[~df.lost, 'latency'].diff().abs().mean(), 6)

print('packet loss rate (%):      ', plr)
print('excessive latency rate (%):', elr)
print('plr + elr (%):             ', plelr)
print('average latency:           ', avg_l)
print('min latency:               ', min_l)
print('max latency:               ', max_l)
print('median latency:            ', mdn_l)
print('latency variance:          ', var_l)
print('latency standard deviation:', std_l)
print('latency jitter:            ', jitter)

packet loss rate (%):       0.1094
excessive latency rate (%): 0.0007
plr + elr (%):              0.1101
average latency:            0.015278
min latency:                0.004967
max latency:                0.100752
median latency:             0.014498
latency variance:           1.5e-05
latency standard deviation: 0.003862
latency jitter:             0.001796


# Statistics

In [13]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            lost_ul = dfs_ul[i][j].lost.sum() if not dfs_ul[i][j].empty else np.nan
            excl_ul = dfs_ul[i][j][~dfs_ul[i][j].lost].excl.sum() if not dfs_ul[i][j].empty else np.nan
            sent_ul = dfs_ul[i][j].seq.count() if not dfs_ul[i][j].empty else np.nan
            lost_dl = dfs_dl[i][j].lost.sum() if not dfs_dl[i][j].empty else np.nan
            excl_dl = dfs_dl[i][j][~dfs_dl[i][j].lost].excl.sum() if not dfs_dl[i][j].empty else np.nan
            sent_dl = dfs_dl[i][j].seq.count() if not dfs_dl[i][j].empty else np.nan
            # print(lost_ul, excl_ul, sent_ul, lost_dl, excl_dl, sent_dl, sep='\t', end='\t')
            row = [*row, lost_ul, excl_ul, sent_ul, lost_dl, excl_dl, sent_dl]
        # print()
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_lost_{schm}', f'ul_excl_{schm}', f'ul_total_{schm}', f'dl_lost_{schm}', f'dl_excl_{schm}', f'dl_total_{schm}']
average = [round(s, 1) for s in np.nanmean(table, axis=0)]
with open("stats_lost_excl.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

In [14]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            plr_ul = round(dfs_ul[i][j].lost.mean() * 100, 4) if not dfs_ul[i][j].empty else np.nan
            elr_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].excl.mean() * 100, 4) if not dfs_ul[i][j].empty else np.nan
            plr_dl = round(dfs_dl[i][j].lost.mean() * 100, 4) if not dfs_dl[i][j].empty else np.nan
            elr_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].excl.mean() * 100, 4) if not dfs_dl[i][j].empty else np.nan
            # print(plr_ul, elr_ul, plr_dl, elr_dl, sep='\t', end='\t')
            row = [*row, plr_ul, elr_ul, plr_dl, elr_dl]
        # print()
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_PLR_{schm}', f'ul_ELR_{schm}', f'dl_PLR_{schm}', f'dl_ELR_{schm}']
average = [round(s, 4) for s in np.nanmean(table, axis=0)]
with open("stats_plr_elr.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

In [15]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            avg_l_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].latency.mean(), 6) if not dfs_ul[i][j].empty else np.nan
            std_l_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].latency.std(), 6) if not dfs_ul[i][j].empty else np.nan
            jitter_ul = round(dfs_ul[i][j].loc[~dfs_ul[i][j].lost, 'latency'].diff().abs().mean(), 6) if not dfs_ul[i][j].empty else np.nan
            avg_l_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].latency.mean(), 6) if not dfs_dl[i][j].empty else np.nan
            std_l_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].latency.std(), 6) if not dfs_dl[i][j].empty else np.nan
            jitter_dl = round(dfs_dl[i][j].loc[~dfs_dl[i][j].lost, 'latency'].diff().abs().mean(), 6) if not dfs_dl[i][j].empty else np.nan
            # print(avg_l_ul, std_l_ul, jitter_ul, avg_l_dl, std_l_dl, jitter_dl, sep='\t', end='\t')
            row = [*row, avg_l_ul, std_l_ul, jitter_ul, avg_l_dl, std_l_dl, jitter_dl]
        # print()    
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_avg_L_{schm}', f'ul_std_L_{schm}', f'ul_jitter_{schm}', f'dl_avg_L_{schm}', f'dl_std_L_{schm}', f'dl_jitter_{schm}']
average = [round(s[1], 6) if i%3 == 1 else round(s[0], 6) for i, s in enumerate(zip(np.nanmean(table, axis=0), nanrms(table, axis=0)))]
with open("stats_latency.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

# Handover Info

In [16]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from collections import namedtuple
from pprint import pprint
import portion as P

# __all__ = [
#     'myQueue',
#     'mi_parse_ho',
#     'cut_head_tail',
#     'get_ho_interval',
#     'label_ho_info',
# ]

class myQueue:
    def __init__(self, maxsize=0):
        self.data = []
        self.maxsize = maxsize if maxsize > 0 else float('inf')
    def tolist(self):
        return self.data
    def size(self):
        return self.maxsize
    def len(self):
        return len(self.data)
    def empty(self):
        return self.len() == 0
    def full(self):
        return self.len() == self.maxsize
    def pop(self, index=0):
        """
        if index > 0, recursively pop() until pop out the specific element.
        return the final popped-out element.
        """
        for _ in range(index, 0, -1):
            self.pop()
        return self.data.pop(0) if not self.empty() else None
    def push(self, element):
        """
        return 0 if success; 1 if the front is popped.
        """
        flag = 0
        if self.full():
            self.pop()
            flag = 1
        self.data.append(element)
        return flag
    def front(self):
        return self.data[0] if not self.empty() else None
    def rear(self):
        return self.data[-1] if not self.empty() else None
    def get(self, index):
        if isinstance(index, list):
            tmp = []
            for i in index:
                tmp = [*tmp, self.get(i)]
            return tmp
        return self.data[index] if index < self.len() and abs(index) <= self.len() else None
    def find(self, element):
        if isinstance(element, list):
            for ele in element:
                index = self.find(ele)
                if index != None:
                    return index
            return None
        return self.data.index(element) if element in self.data else None

def mi_parse_ho(df, tz=0, debug=False):
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
    
    ### Define Basic Element
    HO = namedtuple('HO', 'start, end, cause, others, st_scel', defaults=tuple([None]*4+[0]))
    stNR = namedtuple('stNR', 'snrPCI, tnrPCI', defaults=tuple([None]*2))
    stLTE = namedtuple('stLTE', 'sPCI, sFreq, tPCI, tFreq', defaults=tuple([None]*4))
    NR_CEL = namedtuple('NR_CEL', 'nrPCI, nrFreq', defaults=tuple([None]*2))
    LTE_CEL = namedtuple('LTE_CEL', 'ePCI, ECI, eNB, BID, DL_Freq, DL_BW, UL_Freq, UL_BW', defaults=tuple([None]*8))
    C = namedtuple('C', HO._fields + stLTE._fields + stNR._fields + \
        LTE_CEL._fields + tuple([f'{s}1' for s in LTE_CEL._fields]) + NR_CEL._fields + tuple([f'{s}1' for s in NR_CEL._fields]), 
        defaults=tuple([None]*30))
    
    def dprint(*args, **kwargs):
        if debug:
            print(*args, **kwargs)
    
    def NR_OTA(pos=None):
        row = df.iloc[pos] if pos else df.iloc[i]
        if row.type_id == '5G_NR_RRC_OTA_Packet':
            return True
        else:
            return False
    
    def CEL_INFO(pos=None):
        row = df.iloc[pos] if pos else df.iloc[i]
        if row.type_id == 'LTE_RRC_Serv_Cell_Info':
            return True
        else:
            return False
    
    def nr_track(pos=None):
        row = df.iloc[pos] if pos else df.iloc[i]
        if int(row.PCI) in [0, 65535]:  # 65535 is for samgsung; 0 is for xiaomi.
            return NR_CEL()
        else:
            return NR_CEL(int(row.PCI), int(row.Freq))
    
    def eci_track(pos=None):
        row = df.iloc[pos] if pos else df.iloc[i]
        PCI = int(row['PCI'])
        ECI = int(row['Cell Identity'])
        eNB = ECI // 256
        BID = int(row['Band ID'])
        DL_Freq = int(row['DL frequency'])
        DL_BW = row['DL bandwidth']
        UL_Freq = int(row['UL frequency'])
        UL_BW = row['UL bandwidth']
        return LTE_CEL(PCI, ECI, eNB, BID, DL_Freq, DL_BW, UL_Freq, UL_BW)
    
    def peek_nr(pos=None, look_after=0.5, look_before=0.0):
        ## look_after == 0.5 is a magic number
        ### TODO 先偷看 ho start - end 之間的 cell information
        if pos:  # position of end of an event
            for j in range(i, pos):
                if NR_OTA(j):
                    qpscell.push(nr_track(j))
        ### END TODO
        # dprint(f'pscell={pscell}')
        # dprint(qpscell.tolist())
        index = None
        for j in range(qpscell.len()):
            if pscell != qpscell.get(j):
                index = j
                break
        # dprint(f'index={index}')
        if index != None:
            return qpscell.pop(index)
        ### haven't find pci change yet!
        t = df['Timestamp'].iloc[i]
        for j in range(i, len(df)):  # 往前走，最多走到底
            t1 = df["Timestamp"].iloc[j]
            if (t1 - t).total_seconds() > look_after:
                break
            if df['type_id'].iloc[j] != '5G_NR_RRC_OTA_Packet':
                continue
            row = df.iloc[j]
            if int(row.PCI) in [0, 65535]:  # 65535 is for samgsung; 0 is for xiaomi.
                return NR_CEL()
            else:
                return NR_CEL(int(row.PCI), int(row.Freq))
        return pscell
    
    def peek_eci(pos=None, look_after=0.5, look_before=0.0):
        ## look_after == 0.5 is a magic number
        ### TODO 先偷看 ho start - end 之間的 cell information
        if pos:  # position of end of an event
            for j in range(i, pos):
                if CEL_INFO(j):
                    qpcell.push(eci_track(j))
        ### END TODO
        # dprint(f'pcell={pcell}')
        # dprint(qpcell.tolist())
        index = None
        for j in range(qpcell.len()):
            if pcell != qpcell.get(j):
                index = j
                break
        # dprint(f'index={index}')
        if index != None:
            return qpcell.pop(index)
        ### haven't find pci change yet!
        t = df['Timestamp'].iloc[i]
        for j in range(i, len(df)):  # 往前走，最多走到底
            t1 = df['Timestamp'].iloc[j]
            if (t1 - t).total_seconds() > look_after:
                break
            if df['type_id'].iloc[j] != 'LTE_RRC_Serv_Cell_Info':
                continue
            row = df.iloc[j]
            PCI = int(row['PCI'])
            ECI = int(row['Cell Identity'])
            eNB = ECI // 256
            BID = int(row['Band ID'])
            DL_Freq = int(row['DL frequency'])
            DL_BW = row['DL bandwidth']
            UL_Freq = int(row['UL frequency'])
            UL_BW = row['UL bandwidth']
            return LTE_CEL(PCI, ECI, eNB, BID, DL_Freq, DL_BW, UL_Freq, UL_BW)
        return pcell

    def find_1st_after(target, look_after=1.0):
        for j in range(i, len(df)):  # 往前走，最多走到底
            t1 = df["Timestamp"].iloc[j]
            if (t1 - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] in [1,'1']:
                return t1, j  # timestamp & position
        return None, None

    def find_1st_before(target, look_before=1.0):
        for j in range(i, -1, -1):  # 倒退嚕，最多走回頭
            t1 = df["Timestamp"].iloc[j]
            if (t - t1).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [1,'1']:
                return t1, j  # timestamp & position
        return None, None

    D = {
        ### Conn Setup/Rel & HO
        'Conn_Rel':[],    # Conn Release: rrcConnectionRelease
        'Conn_Setup':[],  # Conn Setup: rrcConnectionRequest + rrcConnectionSetup
        'LTE_HO': [],     # E_PCel -> E_PCel’: lte-rrc.t304 & LTE_PCel does change
        'SN_Rel': [],     # EUTRA + NR -> EUTRA:(CHT) lte-rrc.t304 & LTE_PCel does not change
                          #                     (TWM) nr-Config-r15: release (0) 
        'SN_Setup': [],   # EUTRA -> EUTRA + NR:(CHT) lte-rrc.t304 + nr-rrc.t304 + dualConnectivityPHR: setup (1) & LTE_PCel does not change
                          #                     (TWM) nr-rrc.t304 + dualConnectivityPHR: setup (1)
        'MN_HO': [],      # E_PCel + N_PSCel -> E_PCel’ + N_PSCel: lte-rrc.t304 + nr-rrc.t304 + dualConnectivityPHR: setup (1) & LTE_PCel does change
        'SN_HO': [],      # E_PCel + N_PSCel -> E_PCel + N_PSCel’: nr-rrc.t304
        'MNSN_HO': [],         # (TWM)
        'SN_Rel_MN_HO': [],    # (TWM)
        'SN_Setup_MN_HO': [],  # (TWM)
        ### Link Failure
        'SCG_Failure': [],   # scgFailureInformationNR-r15
        'MCG_Failure': [],   # rrcConnectionReestablishmentRequest + rrcConnectionReestablishmentComplete
        'NAS_Recovery': [],  # rrcConnectionReestablishmentRequest + rrcConnectionReestablishmentReject + rrcConnectionRequest + rrcConnectionSetup
        # MCG_Failure, NAS_Recovery may be caused by 'reconfigurationFailure (0)', 'handoverFailure (1)', 'otherFailure (2)'
        }
    
    A = { 'Conn_Rel':[], 'Conn_Setup':[],
          'LTE_HO': [], 'SN_Rel': [], 'SN_Setup': [], 'MN_HO': [], 'SN_HO': [],
          'MNSN_HO': [], 'SN_Rel_MN_HO': [], 'SN_Setup_MN_HO': [],
          'SCG_Failure': [], 'MCG_Failure': [], 'NAS_Recovery': [] }
    
    qpscell = myQueue(3)
    qpcell = myQueue(3)
    
    init = 1
    pcell, pscell = LTE_CEL(), NR_CEL()
    prev_pci, prev_freq = None, None
    
    for i, row in df.iterrows():
        if NR_OTA():
            qpscell.push(nr_track())
            continue
        elif CEL_INFO():
            qpcell.push(eci_track())
            continue
        if init:
            t_init, pci_init, freq_init = row.Timestamp, int(row.PCI), int(row.Freq)
            pcell = LTE_CEL(ePCI=pci_init, DL_Freq=freq_init)
            dprint(f"{t_init} | Initial PCI={pci_init} EARFCN={freq_init}")
            dprint()
            init = 0
        
        t, pci, freq = row.Timestamp, int(row.PCI), int(row.Freq)
        
        if (prev_pci, prev_freq) != (pci, freq):
            for j in range(i, len(df)):  # 往前走，最多走到底
                if CEL_INFO(j):
                    next_pcell = eci_track(j)
                    if next_pcell[0] == pci:
                        qpcell.push(next_pcell)
                        break
                elif not NR_OTA(j):
                    if df['PCI'].iloc[j] != pci:
                        break
        
        if not qpscell.empty():
            pscell = qpscell.pop()
        if not qpcell.empty():
            pcell = qpcell.pop()
        
        ### Conn_Rel
        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            A['Conn_Rel'].append(C(*HO(start=t), *stLTE(sPCI=pci, sFreq=freq), *stNR(snrPCI=pscell[0]), *pcell, *LTE_CEL(), *pscell, *NR_CEL()))
            dprint(f"{t}, {pd.NaT} | Conn_Rel at PCI={pci} EARFCN={freq}.")
            dprint(f'{tuple(pcell)} -> {tuple(LTE_CEL())}')
            dprint(f'{tuple(pscell)} ->{tuple(NR_CEL())}')
            pcell, pscell = LTE_CEL(), NR_CEL()
            dprint()

        ### Conn_Setup
        if df["rrcConnectionRequest"].iloc[i] == 1:
            a, j1 = find_1st_after('rrcConnectionReconfigurationComplete',look_after=2)
            b, j2 = find_1st_after('securityModeComplete',look_after=2)
            end = a if a > b else b
            j = j1 if a > b else j2
            _pcell = peek_eci(pos=j)
            D['Conn_Setup'].append(HO(start=t, end=end))
            A['Conn_Setup'].append(C(*HO(start=t, end=end), *stLTE(tPCI=pci, tFreq=freq), *stNR(), *pcell, *_pcell, *pscell, *pscell))
            dprint(f"{t}, {end} | Conn_Setup to PCI={pci} EARFCN={freq}.")
            dprint(f'{tuple(pcell)} -> {tuple(_pcell)}')
            dprint(f'{tuple(pscell)} -> {tuple(pscell)}')
            dprint()
        
        ### SN_Setup, SN_Rel, MO_HO, LTE_HO
        if df["lte-rrc.t304"].iloc[i] == 1:
            end, j = find_1st_after('rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = pci, int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = freq, int(df['dl-CarrierFreq'].iloc[i])
            nr_target_cell = int(df["nr_physCellId"].iloc[i])
            
            n = 0
            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others=f'Set up {n} SCell.'
            else:
                others=None
            
            if serv_freq != target_freq:
                others = f'{others} Inter-Freq HO.' if others else 'Inter-Freq HO.'
            
            ### SN_Setup, MN_HO
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                ### SN_Setup
                if serv_cell == target_cell and serv_freq == target_freq:
                    _pscell = peek_nr(pos=j)
                    D['SN_Setup'].append(HO(start=t, end=end, others=others, st_scel=n))
                    A['SN_Setup'].append(C(*HO(start=t, end=end, others=others, st_scel=n), *stLTE(sPCI=serv_cell, sFreq=serv_freq), *stNR(tnrPCI=nr_target_cell), *pcell, *pcell, *pscell, *_pscell))
                    dprint(f"{t}, {end} | SN_Setup to nrPCI={nr_target_cell} | {others}")
                    dprint(f'{tuple(pcell)} -> {tuple(pcell)}')
                    dprint(f'{tuple(pscell)} -> {tuple(_pscell)}')
                    dprint()
                else:
                ### MN_HO
                    _pcell = peek_eci(pos=j)
                    D['MN_HO'].append(HO(start=t, end=end, others=others, st_scel=n))
                    A['MN_HO'].append(C(*HO(start=t, end=end, others=others, st_scel=n), *stLTE(sPCI=serv_cell, sFreq=serv_freq, tPCI=target_cell, tFreq=target_freq), *stNR(snrPCI=pscell[0]), *pcell, *_pcell, *pscell, *pscell))
                    dprint(f"{t}, {end} | MN_HO ({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {others}")
                    dprint(f'{tuple(pcell)} -> {tuple(_pcell)}')
                    dprint(f'{tuple(pscell)} -> {tuple(pscell)}')
                    dprint()
            else:
            ### SN_Rel, LTE_HO
                ### SN_Rel
                if serv_cell == target_cell and serv_freq == target_freq:
                    a, b = find_1st_before("scgFailureInformationNR-r15")
                    if a is not None:
                        others = f'{others} Caused by scg-failure.' if others else 'Caused by scg-failure.'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, st_scel=n))
                    A['SN_Rel'].append(C(*HO(start=t, end=end, others=others, st_scel=n), *stLTE(sPCI=serv_cell, sFreq=serv_freq), *stNR(snrPCI=pscell[0]), *pcell, *pcell, *pscell, *NR_CEL()))
                    dprint(f"{t}, {end} | SN_Rel at nrPCI={pscell[0]} | {others}")
                    dprint(f'{tuple(pcell)} -> {tuple(pcell)}')
                    dprint(f'{tuple(pscell)} -> {tuple(NR_CEL())}')
                    pscell = NR_CEL()
                    dprint()
                else:
                ### LTE_HO
                    _pcell = peek_eci(pos=j)
                    D['LTE_HO'].append(HO(start=t, end=end, others=others, st_scel=n))
                    A['LTE_HO'].append(C(*HO(start=t, end=end, others=others, st_scel=n), *stLTE(sPCI=serv_cell, sFreq=serv_freq, tPCI=target_cell, tFreq=target_freq), *stNR(), *pcell, *_pcell, *pscell, *pscell))
                    dprint(f"{t}, {end} | LTE_HO ({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {others}")
                    dprint(f'{tuple(pcell)} -> {tuple(_pcell)}')
                    dprint(f'{tuple(pscell)} -> {tuple(pscell)}')
                    dprint()

        ### SN_HO
        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
            end, j = find_1st_after('rrcConnectionReconfigurationComplete')
            nr_target_cell = int(df["nr_physCellId"].iloc[i])
            _pscell = peek_nr(pos=j)
            D['SN_HO'].append(HO(start=t, end=end))
            A['SN_HO'].append(C(*HO(start=t, end=end), *stLTE(sPCI=pci, sFreq=freq), *stNR(snrPCI=pscell[0], tnrPCI=nr_target_cell), *pcell, *pcell, *pscell, *_pscell))
            dprint(f"{t}, {end} | SN_HO to nrPCI={nr_target_cell}")
            dprint(f'{tuple(pcell)} -> {tuple(pcell)}')
            dprint(f'{tuple(pscell)} -> {tuple(_pscell)}')
            dprint()

        ### SCG_Failure
        if df["scgFailureInformationNR-r15"].iloc[i] == 1:
            # others = df["failureType-r15"].iloc[i]
            cause = df["failureType-r15"].iloc[i]
            _pscell = peek_nr()
            D['SCG_Failure'].append(HO(start=t, cause=cause))  # end time??
            A['SCG_Failure'].append(C(*HO(start=t, cause=cause), *stLTE(sPCI=pci, sFreq=freq), *stNR(snrPCI=pscell[0]), *pcell, *pcell, *pscell, *_pscell))
            dprint(f"{t}, {pd.NaT} | SCG_Failure at nrPCI={pscell[0]} | {cause}")
            dprint(f'{tuple(pcell)} -> {tuple(pcell)}')
            dprint(f'{tuple(pscell)} -> {tuple(_pscell)}')
            ### SCG Fail 之後必定會 SN Rel
            dprint()
        
        ### MCG_Failure (type II), NAS_Recovery (type III)
        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:
            end1, j1 = find_1st_after('rrcConnectionReestablishmentComplete', look_after=1)
            end2, j2 = find_1st_after('rrcConnectionReestablishmentReject', look_after=1)
            end3, j3 = find_1st_after('rrcConnectionRequest', look_after=1)
            # others = df["reestablishmentCause"].iloc[i]
            cause = df["reestablishmentCause"].iloc[i]
            # target_cell = int(df['physCellId.3'].iloc[i])
            serv_cell, target_cell = pci, int(df['physCellId.3'].iloc[i])
            serv_freq, target_freq = freq, None
            
            ### MCG_Failure (type II)
            if (end1 and not end2) or (end1 and end2 and end1 < end2):
                # dprint(end1, end2)
                end, j = end1, j1
                _pcell = peek_eci()
                D['MCG_Failure'].append(HO(start=t, end=end, cause=cause))
                A['MCG_Failure'].append(C(*HO(start=t, end=end, cause=cause), *stLTE(sPCI=serv_cell, sFreq=serv_freq, tPCI=target_cell, tFreq=target_freq), *stNR(snrPCI=pscell[0]), *pcell, *_pcell, *pscell, *NR_CEL()))
                dprint(f"{t}, {end} | MCG_Failure PCI={serv_cell} -> PCI={target_cell}, recconected to {pci} | {cause}")
                dprint(f'{tuple(pcell)} -> {tuple(_pcell)}')
                dprint(f'{tuple(pscell)} -> {tuple(NR_CEL())}')
                pscell = NR_CEL()
                dprint()
                ### MCG Fail 之後有機會不經過 RRC Connection Setup 就 Reconnect
            else: 
            ### NAS_Recovery (type III)
                # dprint(end1, end2)
                end, j = end3, j3
                _pcell = peek_eci()
                D['NAS_Recovery'].append(HO(start=t, end=end-pd.Timedelta(microseconds=1) if end else None, cause=cause))  # end time??
                A['NAS_Recovery'].append(C(*HO(start=t, end=end-pd.Timedelta(microseconds=1) if end else None, cause=cause), *stLTE(sPCI=serv_cell, sFreq=serv_freq, tPCI=target_cell, tFreq=target_freq), *stNR(snrPCI=pscell[0]), *pcell, *_pcell, *pscell, *NR_CEL()))
                dprint(f"{t}, {end} | NAS_Recovery PCI={serv_cell} -> PCI={target_cell} | {cause}")
                dprint(f'{tuple(pcell)} -> {tuple(_pcell)}')
                dprint(f'{tuple(pscell)} -> {tuple(NR_CEL())}')
                pscell = NR_CEL()
                dprint()
        
        ### Update previous pci, freq
        prev_pci, prev_freq = pci, freq
    
    ### Build DataFrame
    df_HO = pd.DataFrame()
    for key in A.keys():
        df_HO = pd.concat([df_HO, \
            pd.DataFrame(A[key], index=[key]*len(A[key]))])
    if df_HO.empty:
        print("************** Empty DataFrame!! **************")
    df_HO = df_HO.sort_values(by=['start']).reset_index()
    df_HO = df_HO.rename(columns={'index': 'ho_type'})
    df_HO = df_HO.reindex(
        ['start','end','ho_type','intr','sPCI','sFreq','tPCI','tFreq','snrPCI','tnrPCI','cause','others','st_scel'] + \
            df_HO.columns.tolist()[df_HO.columns.get_loc('ePCI'):df_HO.columns.get_loc('nrFreq1')+1], axis=1)
    df_HO['start'] = pd.to_datetime(df_HO['start'])
    df_HO['end'] = pd.to_datetime(df_HO['end'])
    df_HO['Timestamp'] = df_HO['start']
    df_HO['intr'] = (df_HO['end'] - df_HO['start']).dt.total_seconds()
    df_HO['type_id'] = 'RRC_OTA_Handover_Parsing'
    ### Set dtypes
    df_HO['ho_type'] = df_HO['ho_type'].astype('category')
    df_HO['cause'] = df_HO['cause'].astype('category')
    df_HO['others'] = df_HO['others'].astype('string')
    df_HO['st_scel'] = df_HO['st_scel'].astype('Int8')
    df_HO['DL_BW'] = df_HO['DL_BW'].astype('category')
    df_HO['DL_BW1'] = df_HO['DL_BW1'].astype('category')
    df_HO['UL_BW'] = df_HO['UL_BW'].astype('category')
    df_HO['UL_BW1'] = df_HO['UL_BW1'].astype('category')
    for tag in df_HO.columns[df_HO.columns.get_loc('sPCI'):df_HO.columns.get_loc('nrFreq1')+1]:
        if tag not in ['cause','others','DL_BW','DL_BW1','UL_BW','UL_BW1']:
            df_HO[tag] = df_HO[tag].astype('Int32')
    df_HO['intr'] = df_HO['intr'].astype('float32')
    df_HO['Timestamp'] = pd.to_datetime(df_HO['Timestamp'])
    df_HO['type_id'] = df_HO['type_id'].astype('category')
    return df_HO, A, D

def cut_head_tail(df_HO, df, mode='ul'):
    if mode == 'ul':
        start = df.iloc[0].xmit_time
        stop = df.iloc[-1].xmit_time
        df_HO = df_HO.query('Timestamp >= @start & Timestamp <= @stop').copy().reset_index(drop=True)
    if mode == 'dl':
        start = df.iloc[0].arr_time
        stop = df.iloc[-1].arr_time
        df_HO = df_HO.query('Timestamp >= @start & Timestamp <= @stop').copy().reset_index(drop=True)
    return df_HO

def is_disjoint(set1, set2):
    """
    Check if two sets are disjoint.
    """
    return (set1 & set2).empty

def is_disjoint_dict(E):
    test_intv = P.empty()
    for key, val in E.items():
        # print(key)
        for intv in val:
            if is_disjoint(test_intv, intv.interval):
                test_intv = test_intv | intv.interval
            else:
                print(key, intv.index)
                return False
    return True

def interp(x, y, ratio):
    """
    Interpolation

    Args:
        x, y (datetime.datetime): x < y
        ratio (float): a decimal numeral in a range [0, 1]; 0 means break at x, 1 means break at y.
    Returns:
        (datetime.datetime): breakpoint of interpolation
    """
    return x + (y - x) * ratio

def get_ho_interval(df, sec=(1, 3), ratio=0.5,
                 ignored=['Conn_Setup','Conn_Rel'],
                 handover=['LTE_HO','SN_Rel','SN_Setup','MN_HO','SN_HO','MNSN_HO','SN_Rel_MN_HO','SN_Setup_MN_HO'],
                 linkfailure=['SCG_Failure','MCG_Failure','NAS_Recovery']):
    
    HO_INTV = namedtuple('HO_INTV', 'index, interval, state1, state2, st_scel, cause, interrupt, \
                                     ePCI, earfcn, nrPCI, ePCI1, earfcn1, nrPCI1', defaults=tuple([None]*12))
    
    def ignore_col(row):
        if row.ho_type in ignored:
            return False
        else:
            return True
    df = df[df.apply(ignore_col, axis=1)].reset_index(drop=True)
    
    column_names = []
    for type_name in handover + linkfailure:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    E = { col:[] for col in column_names }
    
    for i, row in df.iterrows():
        prior_row = df.iloc[i-1] if i != 0 else None
        post_row = df.iloc[i+1] if i != len(df)-1 else None
        ### peek the next event
        if i != len(df)-1 and pd.notna(row.end) and row.end > post_row.start:
            print(i, row.start, row.end, row.ho_type, row.cause)
            print(i+1, post_row.start, post_row.end, post_row.ho_type, post_row.cause)
            continue
        ### peri_interval
        if pd.isna(row.end):
            peri_interval = P.singleton(row.start)
        else:
            peri_interval = P.closed(row.start, row.end)
        ### prior_interval
        C = row.start - pd.Timedelta(seconds=sec[0]) if row.ho_type in handover else row.start - pd.Timedelta(seconds=sec[1])
        D = row.start
        prior_interval = P.closedopen(C, D)
        if ratio != None and i != 0:
            A = max(prior_row.start, prior_row.end)
            B = max(prior_row.start, prior_row.end) + pd.Timedelta(seconds=sec[0]) if prior_row.ho_type in handover else max(prior_row.start, prior_row.end) + pd.Timedelta(seconds=sec[1])
            if P.openclosed(A, B).overlaps(prior_interval):
                # print("Overlaps with the previous!")
                bkp = interp(C, B, ratio)
                bkp = max(bkp, A)  # to avoid the breakpoint overlaps the previous event's duration
                # bkp = min(max(bkp, A), D)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                prior_interval = P.closedopen(bkp, D)
                if A in prior_interval:
                    prior_interval = P.open(bkp, D)
                # blindly set as open inverval is fine, but may miss one point.
        ### post_interval
        C = row.end
        D = row.end + pd.Timedelta(seconds=sec[0]) if row.ho_type in handover else row.end + pd.Timedelta(seconds=sec[1])
        post_interval = P.openclosed(C, D)
        if ratio != None and i != len(df)-1:
            A = min(post_row.start, post_row.end) - pd.Timedelta(seconds=sec[0]) if post_row.ho_type in handover else min(post_row.start, post_row.end) - pd.Timedelta(seconds=sec[1])
            B = min(post_row.start, post_row.end)
            if P.closedopen(A, B).overlaps(post_interval):
                # print("Overlaps with the following!")
                bkp = interp(A, D, ratio)
                bkp = min(bkp, B)  # to avoid the breakpoint overlaps the following event's duration
                # bkp = max(min(bkp, B), C)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                post_interval = P.open(C, bkp)
        ### append dictionary
        type_name = row.ho_type
        state1, state2 = 'sn_change', 'sn_change'
        if type_name in linkfailure:
            state1, state2 = 'link_failure', 'link_failure'
        if type_name in ['LTE_HO','MN_HO','MNSN_HO','SN_Rel_MN_HO','SN_Setup_MN_HO']:
            state1 = 'inter_freq' if row.sFreq != row.tFreq else 'intra_freq'
            if pd.notna(row.eNB) and pd.notna(row.eNB1) and row.eNB != row.eNB1:
                state2 = 'inter_enb'
            elif row.sPCI != row.tPCI:
                state2 = 'inter_sector'
            elif row.sPCI == row.tPCI:
                state2 = 'intra_sector'
            else:
                print("************** inter_enb, unknown eNB_ID **************")
                state2 = 'inter_enb'
        E[f'before_{type_name}'].append(HO_INTV(i, prior_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        E[f'during_{type_name}'].append(HO_INTV(i, peri_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        E[f'after_{type_name}'].append(HO_INTV(i, post_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        ### check whether the intervals are pairwise disjoint
        if not is_disjoint_dict(E):
            print('Warning: Intervals are not totally disjoint!')
    return E

def label_ho_info(df, E, mode='ul'):
    def removeprefix(string, prefix=['before','during','after']):
        for pref in prefix:
            if string.startswith(pref):
                return pref, string[len(pref)+1:]
        return None, string
    
    df = df.reindex(columns=[*list(df.columns),
            'ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
            'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1'])
    
    df[['ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1']] = \
        [-1, '-', 'stable', 'stable', 'stable', 0, None, None,
         None, None, None, None, None, None]
            
    for key, val in E.items():
        pref, key = removeprefix(key)
        for intv in val:
            if intv.interval.empty:
                continue
            # print(pref, key)
            # print(intv.interval)
            if mode == 'ul':
                df.loc[(df['xmit_time'] >= intv.interval.lower) & (df['xmit_time'] <= intv.interval.upper),
                       ('ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
                        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1')] = \
                        [intv.index, pref, key, intv.state1, intv.state2, intv.st_scel, intv.cause, intv.interrupt,
                        intv.ePCI, intv.earfcn, intv.nrPCI, intv.ePCI1, intv.earfcn1, intv.nrPCI1]
            if mode == 'dl':
                df.loc[(df['arr_time'] >= intv.interval.lower) & (df['arr_time'] <= intv.interval.upper),
                       ('ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
                        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1')] = \
                        [intv.index, pref, key, intv.state1, intv.state2, intv.st_scel, intv.cause, intv.interrupt,
                        intv.ePCI, intv.earfcn, intv.nrPCI, intv.ePCI1, intv.earfcn1, intv.nrPCI1]
    
    df['ho_type0'] = df['ho_type']
    df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type0'] = \
        df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type'] + '_' + df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_cause']
    
    df['_ho_type'] = df['ho_type']
    df['_ho_type0'] = df['ho_type0']
    df['_ho_type1'] = df['ho_type1']
    df['_ho_type2'] = df['ho_type2']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type0'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type0']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type1'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type1']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type2'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type2']
    
    df['ho_index'] = df['ho_index'].astype('Int32')
    df['ho_stage'] = df['ho_stage'].astype('category')
    df['ho_type'] = df['ho_type'].astype('category')
    df['ho_type0'] = df['ho_type0'].astype('category')
    df['ho_type1'] = df['ho_type1'].astype('category')
    df['ho_type2'] = df['ho_type2'].astype('category')
    df['ho_scel'] = df['ho_scel'].astype('Int8')
    df['ho_cause'] = df['ho_cause'].astype('category')
    df['ho_intr'] = df['ho_intr'].astype('float32')
    for tag in df.columns[df.columns.get_loc('ho_ePCI'):df.columns.get_loc('ho_nrPCI1')+1]:
        df[tag] = df[tag].astype('Int32')
    df['_ho_type'] = df['_ho_type'].astype('category')
    df['_ho_type0'] = df['_ho_type'].astype('category')
    df['_ho_type1'] = df['_ho_type1'].astype('category')
    df['_ho_type2'] = df['_ho_type2'].astype('category')
    
    return df

In [17]:
# from handover import mi_parse_ho, get_ho_interval, label_ho_info
i = -1
dfs_ho = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            path = os.path.join(datadir, date, exp, dev, trace, 'data')
            data = os.path.join(path, [s for s in os.listdir(path) if s.startswith('diag_log_') and s.endswith('_rrc.csv')][0])
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            dfs_ho[i].append(df.copy())
            ### add ho info to packet data
            E = get_ho_interval(dfs_ho[i][j].copy())
            dfs_ul[i][j] = label_ho_info(dfs_ul[i][j].copy(), E, mode='ul') if not dfs_ul[i][j].empty else pd.DataFrame()
            dfs_dl[i][j] = label_ho_info(dfs_dl[i][j].copy(), E, mode='dl') if not dfs_dl[i][j].empty else pd.DataFrame()
print(len(dfs_ho))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#02/data/diag_log_qc00_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc02/#02/data/diag_log_qc02_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc03/#02/data/diag_log_qc03_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#03/data/diag_log_qc00_2023-03-15_15-57-16_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc02/#03/data/diag_log_qc02_2023-03-15_15-57-16_rrc.csv True
28 2023-03-15 16:02:32.795056 2023-03-15 16:02:33.203217 MN_HO nan
29 2023-03-15 16:02:33.085107 2023-03-15 16:02:33.161973 MCG_Failure handoverFailure (1)
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc03/#03/data/diag_log_

# Single Radio Analysis

In [166]:
sorter = ['stable',
          'LTE_HO','MN_HO','SN_HO','MNSN_HO','SN_Rel','SN_Setup','SN_Rel_MN_HO','SN_Setup_MN_HO',
          'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
          'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
          'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)',
          'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)']
# mcg failure types: https://www.sharetechnote.com/html/Handbook_LTE_RRC_ConnectionReestablishment.html (0)(1)(2)
# scg failure types: https://www.sharetechnote.com/html/5G/5G_Release17.html (1)(3)

df = pd.DataFrame(columns=['ul_lost','ul_excl','ul_total',
                           'dl_lost','dl_excl','dl_total'],
                  index=sorter).fillna(0).reset_index().rename(columns={'index':'ho_type0'})
display(df)

,ho_type0,ul_lost,ul_excl,ul_total,dl_lost,dl_excl,dl_total
0,stable,0,0,0,0,0,0
1,LTE_HO,0,0,0,0,0,0
2,MN_HO,0,0,0,0,0,0
3,SN_HO,0,0,0,0,0,0
4,MNSN_HO,0,0,0,0,0,0
5,SN_Rel,0,0,0,0,0,0
6,SN_Setup,0,0,0,0,0,0
7,SN_Rel_MN_HO,0,0,0,0,0,0
8,SN_Setup_MN_HO,0,0,0,0,0,0
9,SCG_Failure_t310-Expiry (0),0,0,0,0,0,0


In [182]:
def single_radio_stats(df_main):
    ### Uplink
    if df_main[0].empty:
        table = pd.DataFrame(columns=['ho_type0','ho_count','ul_lost','ul_excl','ul_total'])
    else:
        df_main[0]['ho_type0'] = df_main[0]['ho_type0'].cat.set_categories(sorter)
        ## ul lost
        table = df_main[0].loc[df_main[0]['lost'], ['ho_type0']].value_counts()
        table = pd.DataFrame(table).reset_index().rename(columns={0:'ul_lost'})
        ## ul excl
        table1 = df_main[0].loc[~df_main[0]['lost'] & df_main[0]['excl'], ['ho_type0']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl'})
        table = table.merge(table1, on=['ho_type0'], how='outer') \
            .sort_values(['ho_type0']).reset_index(drop=True).fillna(0)
        ## ul total sent
        table1 = df_main[0].loc[:, ['ho_type0']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_total'})
        table = table.merge(table1, on=['ho_type0'], how='outer') \
            .sort_values(['ho_type0']).reset_index(drop=True).fillna(0)
        # ## ul PLR
        # table['ul_PLR'] = (table['ul_lost'] / table['ul_total'] * 100).round(2)
        # ## ul ELR
        # table['ul_ELR'] = (table['ul_excl'] / (table['ul_total'] - table['ul_lost']) * 100).round(2)
    # display(table)
    
    ### Downlink
    if df_main[1].empty:
        _table = pd.DataFrame(columns=['ho_type0','ho_count','dl_lost','dl_excl','dl_total'])
    else:
        df_main[1]['ho_type0'] = df_main[1]['ho_type0'].cat.set_categories(sorter)
        ## dl lost
        _table = df_main[1].loc[df_main[1]['lost'], ['ho_type0']].value_counts()
        _table = pd.DataFrame(_table).reset_index().rename(columns={0:'dl_lost'})
        ## dl excl
        _table1 = df_main[1].loc[~df_main[1]['lost'] & df_main[1]['excl'], ['ho_type0']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl'})
        _table = _table.merge(_table1, on=['ho_type0'], how='outer') \
            .sort_values(['ho_type0']).reset_index(drop=True).fillna(0)
        ## dl total sent
        _table1 = df_main[1].loc[:, ['ho_type0']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_total'})
        _table = _table.merge(_table1, on=['ho_type0'], how='outer') \
            .sort_values(['ho_type0']).reset_index(drop=True).fillna(0)
        # ## dl PLR
        # _table['dl_PLR'] = (_table['dl_lost'] / _table['dl_total'] * 100).round(2)
        # ## dl ELR
        # _table['dl_ELR'] = (_table['dl_excl'] / (_table['dl_total'] - _table['dl_lost']) * 100).round(2)
    # display(_table)
    
    ### Merge table
    table = table.merge(_table, on=['ho_type0'], how='outer') \
        .sort_values(['ho_type0']).reset_index(drop=True).fillna(0)
    
    ### Count HO
    df = df_main[0].copy() if not df_main[0].empty else df_main[1].copy()
    df['ho_type0'] = df['ho_type0'].cat.set_categories(sorter)
    tb_tmp = df.loc[:, ['ho_type0', 'ho_index']].value_counts().reset_index()
    tb_dict = tb_tmp['ho_type0'].value_counts().to_dict()
    table['ho_count'] = pd.Series(dtype='Int32')
    for key, val in tb_dict.items():
        table.loc[table['ho_type0'] == key, 'ho_count'] = val
    # table = table.fillna(0)

    ## reindex
    table = table.reindex(['ho_type0','ho_count',
                            'ul_lost','ul_excl','ul_total',
                            'dl_lost','dl_excl','dl_total',], axis=1)
    ## set dtypes
    table['ul_lost'] = table['ul_lost'].astype('Int64')
    table['ul_excl'] = table['ul_excl'].astype('Int64')
    table['ul_total'] = table['ul_total'].astype('Int64')
    table['dl_lost'] = table['dl_lost'].astype('Int64')
    table['dl_excl'] = table['dl_excl'].astype('Int64')
    table['dl_total'] = table['dl_total'].astype('Int64')
    table['ho_count'] = table['ho_count'].astype('Int32')
    
    return table

def single_radio_merge(tb_list):
    table = tb_list[0].copy().set_index('ho_type0')
    for i in range(1, len(tb_list)):
        table = table.add(tb_list[i].copy().set_index('ho_type0'), fill_value=0)
    table = table.reset_index()
    return table

In [236]:
N = len(dfs_ho)

# xs = list(it.combinations(range(len(schemes)), 2))
xs = list(it.permutations(range(len(schemes)), 2))
print(xs)

table_list = []
for i in range(N):
# for i in range(16, 17):
    table = pd.DataFrame(columns=['ho_count',
                                  'ul_lost','ul_excl','ul_total',
                                  'dl_lost','dl_excl','dl_total'],
                         index=sorter).fillna(0).reset_index().rename(columns={'index':'ho_type0'})
    for x in xs:
        # print(i)
        # print(x[0], x[1])
        df_main = (dfs_ul[i][x[0]].copy(), dfs_dl[i][x[0]].copy())
        df_subr = (dfs_ul[i][x[1]].copy(), dfs_dl[i][x[1]].copy())
        ### Single Radio
        _table = single_radio_stats(df_main)
        table = single_radio_merge([table, _table])
        ### TODO: Add list to log tables under different schemes
    table_list.append(table)

print(len(table_list))
table_sr = single_radio_merge(table_list)
table_sr['ul_PLR'] = (table_sr['ul_lost'] / (table_sr['ul_total'] + 1e-9) * 100).round(3)
table_sr['ul_ELR'] = (table_sr['ul_excl'] / ((table_sr['ul_total'] + 1e-9) - table_sr['ul_lost']) * 100).round(3)
table_sr['dl_PLR'] = (table_sr['dl_lost'] / (table_sr['dl_total'] + 1e-9) * 100).round(3)
table_sr['dl_ELR'] = (table_sr['dl_excl'] / ((table_sr['dl_total'] + 1e-9) - table_sr['dl_lost']) * 100).round(3)
table_sr = table_sr.reindex(['ho_type0','ho_count',
                             'ul_lost','ul_excl','ul_total','dl_lost','dl_excl','dl_total',
                             'ul_PLR','ul_ELR','dl_PLR','dl_ELR',], axis=1)
table_sr['ho_count'] = table_sr['ho_count'].astype('Int32')

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
20


## Result

In [207]:
display(table_sr)
# display(table_sr.query('ho_count > 0').reset_index(drop=True))
# display(table_sr.query('ho_count > 0').sort_values(by=['ul_PLR','dl_PLR','ul_ELR','dl_ELR'], ascending=False).reset_index(drop=True))
# display(table_sr.query('ho_count > 0').sort_values(by=['dl_PLR','ul_PLR','dl_ELR','ul_ELR'], ascending=False).reset_index(drop=True))

,ho_type0,ho_count,ul_lost,ul_excl,ul_total,dl_lost,dl_excl,dl_total,ul_PLR,ul_ELR,dl_PLR,dl_ELR
0,stable,120,3894,52804,10712426,14048,17918,10714170,0.04,0.49,0.13,0.17
1,LTE_HO,790,398,1926,452010,1024,16160,455924,0.09,0.43,0.22,3.55
2,MN_HO,1328,1406,11234,853324,12620,4520,830546,0.16,1.32,1.52,0.55
3,SN_HO,1298,312,26028,822626,4008,5060,823184,0.04,3.17,0.49,0.62
4,MNSN_HO,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
5,SN_Rel,18,2,148,2106,112,0,1320,0.09,7.03,8.48,0.0
6,SN_Setup,350,2966,14408,135422,23630,2532,130460,2.19,10.88,18.11,2.37
7,SN_Rel_MN_HO,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
8,SN_Setup_MN_HO,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
9,SCG_Failure_t310-Expiry (0),4,0,0,6000,1718,38,5560,0.0,0.0,30.9,0.99


# Dual Radio Analysis

In [197]:
print(len(sorter))
# sorter1 = list(it.permutations(sorter, 2))
sorter1 = list(it.product(sorter, repeat=2))
sorter1 = ['+'.join([s[0],s[1]]) for s in sorter1]
print(sorter1)

22
['stable+stable', 'stable+LTE_HO', 'stable+MN_HO', 'stable+SN_HO', 'stable+MNSN_HO', 'stable+SN_Rel', 'stable+SN_Setup', 'stable+SN_Rel_MN_HO', 'stable+SN_Setup_MN_HO', 'stable+SCG_Failure_t310-Expiry (0)', 'stable+SCG_Failure_randomAccessProblem (1)', 'stable+SCG_Failure_rlc-MaxNumRetx (2)', 'stable+SCG_Failure_synchReconfigFailureSCG (3)', 'stable+SCG_Failure_scg-ReconfigFailure (4)', 'stable+SCG_Failure_srb3-IntegrityFailure (5)', 'stable+SCG_Failure_other-r16 (6)', 'stable+MCG_Failure_reconfigurationFailure (0)', 'stable+MCG_Failure_handoverFailure (1)', 'stable+MCG_Failure_otherFailure (2)', 'stable+NAS_Recovery_reconfigurationFailure (0)', 'stable+NAS_Recovery_handoverFailure (1)', 'stable+NAS_Recovery_otherFailure (2)', 'LTE_HO+stable', 'LTE_HO+LTE_HO', 'LTE_HO+MN_HO', 'LTE_HO+SN_HO', 'LTE_HO+MNSN_HO', 'LTE_HO+SN_Rel', 'LTE_HO+SN_Setup', 'LTE_HO+SN_Rel_MN_HO', 'LTE_HO+SN_Setup_MN_HO', 'LTE_HO+SCG_Failure_t310-Expiry (0)', 'LTE_HO+SCG_Failure_randomAccessProblem (1)', 'LTE_HO+

In [198]:
def dual_radio_stats(df_main, df_subr):
    dfs = [pd.DataFrame(), pd.DataFrame()]
    
    ### Uplink
    if df_main[0].empty:
        table = pd.DataFrame(columns=['ho_type0_m','ho_type0_s','ho_count',
                            'ul_lost_m','ul_lost_ms','ul_lost_mlss','ul_excl_m','ul_excl_ms','ul_excl_mess','ul_total',
                            'ho_type0_ms'])
    else:
        dfs[0] = pd.merge(df_main[0], df_subr[0], on=['seq'], suffixes=('_m','_s')).copy()
        dfs[0]['ho_type0_m'] = dfs[0]['ho_type0_m'].cat.set_categories(sorter)
        dfs[0]['ho_type0_s'] = dfs[0]['ho_type0_s'].cat.set_categories(sorter)
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_m'].astype('string') + '+' + dfs[0]['ho_type0_s'].astype('string')
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_ms'].astype('category')
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_ms'].cat.set_categories(sorter1)
        # display(dfs[0])
        ## main ul lost
        table = dfs[0].loc[dfs[0]['lost_m'], ['ho_type0_ms']].value_counts()
        table = pd.DataFrame(table).reset_index().rename(columns={0:f'ul_lost_m'})
        ## system ul lost
        table1 = dfs[0].loc[dfs[0]['lost_m'] & dfs[0]['lost_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'ul_lost_ms'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul: main lost, subr safe
        table1 = dfs[0].loc[dfs[0]['lost_m'] & ~dfs[0]['lost_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'ul_lost_mlss'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## main ul excl
        table1 = dfs[0].loc[~dfs[0]['lost_m'] & dfs[0]['excl_m'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_m'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## system ul excl
        table1 = dfs[0].loc[(~dfs[0]['lost_m'] & dfs[0]['excl_m']) & (dfs[0]['excl_s']), ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_ms'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul: main excl (main not lost), subr safe (subr not lost)
        table1 = dfs[0].loc[(~dfs[0]['lost_m'] & dfs[0]['excl_m']) & (~dfs[0]['excl_s']), ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_mess'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul total sent
        table1 = dfs[0].loc[:, ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_total'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        # ## ul PLR
        # table['ul_PLR_m'] = (table['ul_lost_m'] / table['ul_total'] * 100).round(2)
        # table['ul_PLR_ms'] = (table['ul_lost_ms'] / table['ul_total'] * 100).round(2)
        # table['ul_PLR_mlss'] = (table['ul_lost_mlss'] / table['ul_total'] * 100).round(2)
        # ## ul ELR
        # table['ul_ELR_m'] = (table['ul_excl_m'] / (table['ul_total'] - table['ul_lost_ms']) * 100).round(2)
        # table['ul_ELR_ms'] = (table['ul_excl_ms'] / (table['ul_total'] - table['ul_lost_ms']) * 100).round(2)
        # table['ul_ELR_mess'] = (table['ul_excl_mess'] / (table['ul_total'] - table['ul_lost_mlss']) * 100).round(2)
    # display(table.drop(columns=['ho_type0_ms']))
    
    ### Downlink
    if df_main[1].empty:
        _table = pd.DataFrame(columns=['ho_type0_m','ho_type0_s','ho_count',
                            'dl_lost_m','dl_lost_ms','dl_lost_mlss','dl_excl_m','dl_excl_ms','dl_excl_mess','dl_total',
                            'ho_type0_ms'])
    else:
        dfs[1] = pd.merge(df_main[1], df_subr[1], on=['seq'], suffixes=('_m','_s')).copy()
        dfs[1]['ho_type0_m'] = dfs[1]['ho_type0_m'].cat.set_categories(sorter)
        dfs[1]['ho_type0_s'] = dfs[1]['ho_type0_s'].cat.set_categories(sorter)
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_m'].astype('string') + '+' + dfs[1]['ho_type0_s'].astype('string')
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_ms'].astype('category')
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_ms'].cat.set_categories(sorter1)
        ## main dl lost
        _table = dfs[1].loc[dfs[1]['lost_m'], ['ho_type0_ms']].value_counts()
        _table = pd.DataFrame(_table).reset_index().rename(columns={0:f'dl_lost_m'})
        ## system dl lost
        _table1 = dfs[1].loc[dfs[1]['lost_m'] & dfs[1]['lost_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:f'dl_lost_ms'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl: main lost, subr safe
        _table1 = dfs[1].loc[dfs[1]['lost_m'] & ~dfs[1]['lost_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:f'dl_lost_mlss'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## main dl excl
        _table1 = dfs[1].loc[~dfs[1]['lost_m'] & dfs[1]['excl_m'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_m'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## system dl excl
        _table1 = dfs[1].loc[(~dfs[1]['lost_m'] & dfs[1]['excl_m']) & (dfs[1]['excl_s']), ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_ms'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl: main excl (main not lost), subr safe (subr not lost)
        _table1 = dfs[1].loc[(~dfs[1]['lost_m'] & dfs[1]['excl_m']) & (~dfs[1]['excl_s']), ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_mess'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl total sent
        _table1 = dfs[1].loc[:, ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_total'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        # ## dl PLR
        # _table['dl_PLR_m'] = (_table['dl_lost_m'] / _table['dl_total'] * 100).round(2)
        # _table['dl_PLR_ms'] = (_table['dl_lost_ms'] / _table['dl_total'] * 100).round(2)
        # _table['dl_PLR_mlss'] = (_table['dl_lost_mlss'] / _table['dl_total'] * 100).round(2)
        # ## dl ELR
        # _table['dl_ELR_m'] = (_table['dl_excl_m'] / (_table['dl_total'] - _table['dl_lost_ms']) * 100).round(2)
        # _table['dl_ELR_ms'] = (_table['dl_excl_ms'] / (_table['dl_total'] - _table['dl_lost_ms']) * 100).round(2)
        # _table['dl_ELR_mess'] = (_table['dl_excl_mess'] / (_table['dl_total'] - _table['dl_lost_mlss']) * 100).round(2)
    # display(_table.drop(columns=['ho_type0_ms']))

    ### Merge table
    table = table.merge(_table, on=['ho_type0_ms'], how='outer') \
        .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
    
    ### Count HO
    df = dfs[0].copy() if not dfs[0].empty else dfs[1].copy()
    df['ho_type0_ms'] = df['ho_type0_m'].astype('string') + '+' + df['ho_type0_s'].astype('string')
    df['ho_type0_ms'] = df['ho_type0_ms'].astype('category')
    df['ho_type0_ms'] = df['ho_type0_ms'].cat.set_categories(sorter1)
    tb_tmp = df.loc[:, ['ho_type0_ms','ho_index_m','ho_index_s']].value_counts().reset_index()
    tb_dict = tb_tmp['ho_type0_ms'].value_counts().to_dict()
    table['ho_count'] = pd.Series(dtype='Int32')
    for key, val in tb_dict.items():
        table.loc[table['ho_type0_ms'] == key, 'ho_count'] = val
    # table = table.fillna(0)
    
    ## restore ho_type0_m & ho_type0_s
    table['ho_type0_m'] = [s[0] for s in table['ho_type0_ms'].str.split('+', n=1, expand=False)]
    table['ho_type0_s'] = [s[1] for s in table['ho_type0_ms'].str.split('+', n=1, expand=False)]

    ## sort by ho_type0_ms
    table['ho_type0_ms'] = table['ho_type0_ms'].cat.set_categories(sorter1)
    table = table.sort_values(['ho_type0_ms'])
    
    ## reindex
    table = table.reindex(['ho_type0_m','ho_type0_s','ho_count',
                            'ul_lost_m','ul_lost_ms','ul_lost_mlss','ul_excl_m','ul_excl_ms','ul_excl_mess','ul_total',
                            'dl_lost_m','dl_lost_ms','dl_lost_mlss','dl_excl_m','dl_excl_ms','dl_excl_mess','dl_total',
                            'ho_type0_ms'], axis=1)
    ## set dtypes
    table['ul_lost_m'] = table['ul_lost_m'].astype('Int64')
    table['ul_lost_ms'] = table['ul_lost_ms'].astype('Int64')
    table['ul_lost_mlss'] = table['ul_lost_mlss'].astype('Int64')
    table['ul_excl_m'] = table['ul_excl_m'].astype('Int64')
    table['ul_excl_ms'] = table['ul_excl_ms'].astype('Int64')
    table['ul_excl_mess'] = table['ul_excl_mess'].astype('Int64')
    table['ul_total'] = table['ul_total'].astype('Int64')
    table['dl_lost_m'] = table['dl_lost_m'].astype('Int64')
    table['dl_lost_ms'] = table['dl_lost_ms'].astype('Int64')
    table['dl_lost_mlss'] = table['dl_lost_mlss'].astype('Int64')
    table['dl_excl_m'] = table['dl_excl_m'].astype('Int64')
    table['dl_excl_ms'] = table['dl_excl_ms'].astype('Int64')
    table['dl_excl_mess'] = table['dl_excl_mess'].astype('Int64')
    table['dl_total'] = table['dl_total'].astype('Int64')
    table['ho_count'] = table['ho_count'].astype('Int32')
    
    return table

def dual_radio_merge(tb_list):
    table = tb_list[0].copy().set_index(['ho_type0_m','ho_type0_s','ho_type0_ms'])
    for i in range(1, len(tb_list)):
        table = table.add(tb_list[i].copy().set_index(['ho_type0_m','ho_type0_s','ho_type0_ms']), fill_value=0)
    table = table.reset_index()
    return table

In [235]:
N = len(dfs_ho)

# xs = list(it.combinations(range(len(schemes)), 2))
xs = list(it.permutations(range(len(schemes)), 2))
print(xs)

table_list = []
for i in range(N):
# for i in range(16, 17):
    sorter2 = [s.split('+')[0] for s in sorter1]
    sorter3 = [s.split('+')[1] for s in sorter1]
    table = pd.DataFrame(columns=['ho_count',
                                'ul_lost_m','ul_lost_ms','ul_lost_mlss','ul_excl_m','ul_excl_ms','ul_excl_mess','ul_total',
                                'dl_lost_m','dl_lost_ms','dl_lost_mlss','dl_excl_m','dl_excl_ms','dl_excl_mess','dl_total',],
                        index=[sorter2, sorter3, sorter1]).fillna(0).reset_index(). \
                        rename(columns={'level_0':'ho_type0_m', 'level_1':'ho_type0_s', 'level_2':'ho_type0_ms'})
    for x in xs:
        # print(i)
        # print(x[0], x[1])
        df_main = (dfs_ul[i][x[0]].copy(), dfs_dl[i][x[0]].copy())
        df_subr = (dfs_ul[i][x[1]].copy(), dfs_dl[i][x[1]].copy())
        ### Dual Radio
        _table = dual_radio_stats(df_main, df_subr)
        # display(_table)
        table = dual_radio_merge([table, _table])
        # display(table)
        ### TODO: Add list to log tables under different schemes
    table_list.append(table)

print(len(table_list))
table_dr = dual_radio_merge(table_list)
## ul PLR
table_dr['ul_PLR_m'] = (table_dr['ul_lost_m'] / (table_dr['ul_total'] + 1e-9) * 100).round(5)
table_dr['ul_PLR_ms'] = (table_dr['ul_lost_ms'] / (table_dr['ul_total'] + 1e-9) * 100).round(5)
table_dr['ul_PLR_mlss'] = (table_dr['ul_lost_mlss'] / (table_dr['ul_total'] + 1e-9) * 100).round(5)
## ul ELR
table_dr['ul_ELR_m'] = (table_dr['ul_excl_m'] / ((table_dr['ul_total'] + 1e-9) - table_dr['ul_lost_ms']) * 100).round(5)
table_dr['ul_ELR_ms'] = (table_dr['ul_excl_ms'] / ((table_dr['ul_total'] + 1e-9) - table_dr['ul_lost_ms']) * 100).round(5)
table_dr['ul_ELR_mess'] = (table_dr['ul_excl_mess'] / ((table_dr['ul_total'] + 1e-9) - table_dr['ul_lost_mlss']) * 100).round(5)
## dl PLR
table_dr['dl_PLR_m'] = (table_dr['dl_lost_m'] / (table_dr['dl_total'] + 1e-9) * 100).round(5)
table_dr['dl_PLR_ms'] = (table_dr['dl_lost_ms'] / (table_dr['dl_total'] + 1e-9) * 100).round(5)
table_dr['dl_PLR_mlss'] = (table_dr['dl_lost_mlss'] / (table_dr['dl_total'] + 1e-9) * 100).round(5)
## dl ELR
table_dr['dl_ELR_m'] = (table_dr['dl_excl_m'] / ((table_dr['dl_total'] + 1e-9) - table_dr['dl_lost_ms']) * 100).round(5)
table_dr['dl_ELR_ms'] = (table_dr['dl_excl_ms'] / ((table_dr['dl_total'] + 1e-9) - table_dr['dl_lost_ms']) * 100).round(5)
table_dr['dl_ELR_mess'] = (table_dr['dl_excl_mess'] / ((table_dr['dl_total'] + 1e-9) - table_dr['dl_lost_mlss']) * 100).round(5)

table_dr = table_dr.reindex(['ho_type0_m','ho_type0_s','ho_count',
                             'ul_lost_m','ul_lost_ms','ul_lost_mlss','ul_excl_m','ul_excl_ms','ul_excl_mess','ul_total',
                             'dl_lost_m','dl_lost_ms','dl_lost_mlss','dl_excl_m','dl_excl_ms','dl_excl_mess','dl_total',
                             'ul_PLR_m','ul_PLR_ms','ul_PLR_mlss','ul_ELR_m','ul_ELR_ms','ul_ELR_mess',
                             'dl_PLR_m','dl_PLR_ms','dl_PLR_mlss','dl_ELR_m','dl_ELR_ms','dl_ELR_mess',
                             'ho_type0_ms'], axis=1)
table_dr['ho_count'] = table_dr['ho_count'].astype('Int32')
table_dr['ho_type0_ms'] = table_dr['ho_type0_ms'].astype('category')
table_dr['ho_type0_ms'] = table_dr['ho_type0_ms'].cat.set_categories(sorter1)
table_dr = table_dr.sort_values(['ho_type0_ms']).reset_index(drop=True)

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]
20


## Result

In [206]:
display(table_dr)
# with pd.option_context('display.max_rows', None):
#     display(table_dr.query('ho_count > 0').reset_index(drop=True))
    # display(table_dr.query('ho_count > 0').sort_values(by=['ul_PLR_m','dl_PLR_m','ul_ELR_m','dl_ELR_m'], ascending=False).reset_index(drop=True))
    # display(table_dr.query('ho_count > 0').sort_values(by=['dl_PLR_m','ul_PLR_m','dl_ELR_m','ul_ELR_m'], ascending=False).reset_index(drop=True))

,ho_type0_m,ho_type0_s,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,ul_total,dl_lost_m,dl_lost_ms,dl_lost_mlss,dl_excl_m,dl_excl_ms,dl_excl_mess,dl_total,ul_PLR_m,ul_PLR_ms,ul_PLR_mlss,ul_ELR_m,ul_ELR_ms,ul_ELR_mess,dl_PLR_m,dl_PLR_ms,dl_PLR_mlss,dl_ELR_m,dl_ELR_ms,dl_ELR_mess,ho_type0_ms
0,stable,stable,120,2615,8,2607,42772,7184,35588,9278660,8535,36,8499,10849,288,10561,9297442,0.03,0.0,0.03,0.46,0.08,0.38,0.09,0.0,0.09,0.12,0.0,0.11,stable+stable
1,stable,LTE_HO,518,376,0,376,1433,0,1433,147001,1665,0,1665,1565,93,1472,149691,0.26,0.0,0.26,0.97,0.0,0.98,1.11,0.0,1.11,1.05,0.06,0.99,stable+LTE_HO
2,stable,MN_HO,1108,270,2,268,1520,1011,509,581774,745,91,654,1775,0,1775,558089,0.05,0.0,0.05,0.26,0.17,0.09,0.13,0.02,0.12,0.32,0.0,0.32,stable+MN_HO
3,stable,SN_HO,1052,487,1,486,5460,2992,2468,502259,1849,0,1849,2268,160,2108,519007,0.1,0.0,0.1,1.09,0.6,0.49,0.36,0.0,0.36,0.44,0.03,0.41,stable+SN_HO
4,stable,MNSN_HO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stable+MNSN_HO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,NAS_Recovery_otherFailure (2),MCG_Failure_handoverFailure (1),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAS_Recovery_otherFailure (2)+MCG_Failure_hand...
480,NAS_Recovery_otherFailure (2),MCG_Failure_otherFailure (2),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAS_Recovery_otherFailure (2)+MCG_Failure_othe...
481,NAS_Recovery_otherFailure (2),NAS_Recovery_reconfigurationFailure (0),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAS_Recovery_otherFailure (2)+NAS_Recovery_rec...
482,NAS_Recovery_otherFailure (2),NAS_Recovery_handoverFailure (1),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAS_Recovery_otherFailure (2)+NAS_Recovery_han...


# Insight

In [240]:
print('Single Radio Statistics: sort by UL PLR, ELR')
table = table_sr.query('ho_count > 0').sort_values(by=['ul_PLR','ul_ELR'], ascending=False). \
    reset_index(drop=True).drop(columns=['dl_lost','dl_excl','dl_total','dl_PLR','dl_ELR'])
display(table)

with pd.option_context('display.max_rows', None):
    print('Dual Radio Statistics: sort by UL PLR_sys, ELR_sys')
    table = table_dr.query('ho_count > 0').sort_values(by=['ul_PLR_ms','ul_ELR_ms'], ascending=False). \
        reset_index(drop=True).drop(columns=['ho_type0_ms',
                                              'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                                              'dl_excl_m','dl_excl_ms','dl_excl_mess',
                                              'dl_total',
                                              'dl_PLR_m','dl_PLR_ms','dl_PLR_mlss',
                                              'dl_ELR_m','dl_ELR_ms','dl_ELR_mess']).head(20)
    display(table)
    
    print('Dual Radio Statistics: sort by UL PLR_mlss (main lost, sub safe), ELR_mess (main excl, sub safe)')
    table = table_dr.query('ho_count > 0').sort_values(by=['ul_PLR_mlss','ul_ELR_mess'], ascending=False). \
        reset_index(drop=True).drop(columns=['ho_type0_ms',
                                              'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                                              'dl_excl_m','dl_excl_ms','dl_excl_mess',
                                              'dl_total',
                                              'dl_PLR_m','dl_PLR_ms','dl_PLR_mlss',
                                              'dl_ELR_m','dl_ELR_ms','dl_ELR_mess']).head(20)
    display(table)

Single Radio Statistics: sort by UL PLR, ELR


,ho_type0,ho_count,ul_lost,ul_excl,ul_total,ul_PLR,ul_ELR
0,NAS_Recovery_otherFailure (2),2,880,816,1770,49.718,91.685
1,SN_Setup,350,2966,14408,135422,2.19,10.878
2,MCG_Failure_otherFailure (2),190,3030,18864,206306,1.469,9.28
3,MN_HO,1328,1406,11234,853324,0.165,1.319
4,SN_Rel,18,2,148,2106,0.095,7.034
5,LTE_HO,790,398,1926,452010,0.088,0.426
6,SN_HO,1298,312,26028,822626,0.038,3.165
7,stable,120,3894,52804,10712426,0.036,0.493
8,MCG_Failure_handoverFailure (1),14,4,6026,17844,0.022,33.778
9,SCG_Failure_synchReconfigFailureSCG (3),12,0,86,510,0.0,16.863


Dual Radio Statistics: sort by UL PLR_sys, ELR_sys


,ho_type0_m,ho_type0_s,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,ul_total,ul_PLR_m,ul_PLR_ms,ul_PLR_mlss,ul_ELR_m,ul_ELR_ms,ul_ELR_mess
0,SN_Setup,SN_Setup,20,62,4,58,1468,150,1318,5792,1.07044,0.06906,1.00138,25.36282,2.59157,22.9857
1,MCG_Failure_otherFailure (2),SN_Setup,20,8,2,6,1311,159,1152,4489,0.17821,0.04455,0.13366,29.21774,3.54357,25.69708
2,SN_Setup,MCG_Failure_otherFailure (2),20,104,2,102,442,116,326,4489,2.31677,0.04455,2.27222,9.85068,2.58525,7.43105
3,stable,SN_Setup,219,125,10,115,273,0,273,68495,0.1825,0.0146,0.1679,0.39863,0.0,0.39924
4,SN_Setup,stable,219,1897,10,1887,8873,0,8873,68495,2.76955,0.0146,2.75495,12.95612,0.0,13.32122
5,SN_HO,MCG_Failure_otherFailure (2),54,8,2,6,426,350,76,21726,0.03682,0.00921,0.02762,1.96096,1.61112,0.34991
6,MCG_Failure_otherFailure (2),SN_HO,54,836,2,834,1578,180,1398,21726,3.84792,0.00921,3.83872,7.26386,0.82858,6.69156
7,MCG_Failure_otherFailure (2),stable,161,1924,3,1921,12947,1128,11819,118431,1.62457,0.00253,1.62204,10.93238,0.95248,10.14419
8,stable,MCG_Failure_otherFailure (2),161,19,3,16,1337,1127,210,118431,0.01604,0.00253,0.01351,1.12896,0.95163,0.17734
9,MN_HO,stable,1108,1083,2,1081,9707,1025,8682,581774,0.18615,0.00034,0.18581,1.66852,0.17619,1.49511


Dual Radio Statistics: sort by UL PLR_mlss (main lost, sub safe), ELR_mess (main excl, sub safe)


,ho_type0_m,ho_type0_s,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,ul_total,ul_PLR_m,ul_PLR_ms,ul_PLR_mlss,ul_ELR_m,ul_ELR_ms,ul_ELR_mess
0,NAS_Recovery_otherFailure (2),LTE_HO,1,197,0,197,86,0,86,320,61.5625,0.0,61.5625,26.875,0.0,69.9187
1,NAS_Recovery_otherFailure (2),SN_HO,1,125,0,125,86,1,85,248,50.40323,0.0,50.40323,34.67742,0.40323,69.10569
2,NAS_Recovery_otherFailure (2),stable,2,558,0,558,644,0,644,1202,46.42263,0.0,46.42263,53.57737,0.0,100.0
3,MCG_Failure_otherFailure (2),SN_HO,54,836,2,834,1578,180,1398,21726,3.84792,0.00921,3.83872,7.26386,0.82858,6.69156
4,SN_Setup,SN_HO,84,563,0,563,1206,31,1175,19791,2.84473,0.0,2.84473,6.09368,0.15664,6.11088
5,SN_Setup,stable,219,1897,10,1887,8873,0,8873,68495,2.76955,0.0146,2.75495,12.95612,0.0,13.32122
6,SN_Setup,MCG_Failure_otherFailure (2),20,104,2,102,442,116,326,4489,2.31677,0.04455,2.27222,9.85068,2.58525,7.43105
7,SN_Rel,SN_Setup,1,1,0,1,27,0,27,54,1.85185,0.0,1.85185,50.0,0.0,50.9434
8,MCG_Failure_otherFailure (2),stable,161,1924,3,1921,12947,1128,11819,118431,1.62457,0.00253,1.62204,10.93238,0.95248,10.14419
9,SN_Setup,MCG_Failure_handoverFailure (1),2,1,0,1,0,0,0,89,1.1236,0.0,1.1236,0.0,0.0,0.0


In [241]:
print('Single Radio Statistics: sort by DL PLR, ELR')
table = table_sr.query('ho_count > 0').sort_values(by=['dl_PLR','dl_ELR'], ascending=False). \
    reset_index(drop=True).drop(columns=['ul_lost','ul_excl','ul_total','ul_PLR','ul_ELR'])
display(table)

with pd.option_context('display.max_rows', None):
    print('Dual Radio Statistics: sort by DL PLR_sys, ELR_sys')
    table = table_dr.query('ho_count > 0').sort_values(by=['dl_PLR_ms','dl_ELR_ms'], ascending=False). \
        reset_index(drop=True).drop(columns=['ho_type0_ms',
                                              'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                                              'ul_excl_m','ul_excl_ms','ul_excl_mess',
                                              'ul_total',
                                              'ul_PLR_m','ul_PLR_ms','ul_PLR_mlss',
                                              'ul_ELR_m','ul_ELR_ms','ul_ELR_mess']).head(20)
    display(table)
    
    print('Dual Radio Statistics: sort by DL PLR_mlss (main lost, sub safe), ELR_mess (main excl, sub safe)')
    table = table_dr.query('ho_count > 0').sort_values(by=['dl_PLR_mlss','dl_ELR_mess'], ascending=False). \
        reset_index(drop=True).drop(columns=['ho_type0_ms',
                                              'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                                              'ul_excl_m','ul_excl_ms','ul_excl_mess',
                                              'ul_total',
                                              'ul_PLR_m','ul_PLR_ms','ul_PLR_mlss',
                                              'ul_ELR_m','ul_ELR_ms','ul_ELR_mess']).head(20)
    display(table)

Single Radio Statistics: sort by DL PLR, ELR


,ho_type0,ho_count,dl_lost,dl_excl,dl_total,dl_PLR,dl_ELR
0,NAS_Recovery_otherFailure (2),2,1526,0,1768,86.312,0.0
1,MCG_Failure_handoverFailure (1),14,6762,46,15964,42.358,0.5
2,SCG_Failure_t310-Expiry (0),4,1718,38,5560,30.899,0.989
3,SN_Setup,350,23630,2532,130460,18.113,2.37
4,MCG_Failure_otherFailure (2),190,30182,10966,204936,14.728,6.275
5,SN_Rel,18,112,0,1320,8.485,0.0
6,SCG_Failure_synchReconfigFailureSCG (3),12,24,132,346,6.936,40.994
7,MN_HO,1328,12620,4520,830546,1.519,0.553
8,SN_HO,1298,4008,5060,823184,0.487,0.618
9,LTE_HO,790,1024,16160,455924,0.225,3.552


Dual Radio Statistics: sort by DL PLR_sys, ELR_sys


,ho_type0_m,ho_type0_s,ho_count,dl_lost_m,dl_lost_ms,dl_lost_mlss,dl_excl_m,dl_excl_ms,dl_excl_mess,dl_total,dl_PLR_m,dl_PLR_ms,dl_PLR_mlss,dl_ELR_m,dl_ELR_ms,dl_ELR_mess
0,MCG_Failure_otherFailure (2),MCG_Failure_handoverFailure (1),2,602,439,163,60,44,16,1619,37.18345,27.1155,10.06794,5.08475,3.72881,1.0989
1,MCG_Failure_handoverFailure (1),MCG_Failure_otherFailure (2),2,1040,439,601,23,23,0,1619,64.23718,27.1155,37.12168,1.94915,1.94915,0.0
2,stable,SCG_Failure_t310-Expiry (0),4,440,440,0,158,158,0,2099,20.96236,20.96236,0.0,9.52381,9.52381,0.0
3,SCG_Failure_t310-Expiry (0),stable,4,1149,440,709,1,1,0,2099,54.74035,20.96236,33.77799,0.06028,0.06028,0.0
4,SN_Setup,MCG_Failure_otherFailure (2),20,1049,608,441,95,0,95,4300,24.39535,14.13953,10.25581,2.57313,0.0,2.46178
5,MCG_Failure_otherFailure (2),SN_Setup,20,1607,608,999,0,0,0,4300,37.37209,14.13953,23.23256,0.0,0.0,0.0
6,MCG_Failure_otherFailure (2),MCG_Failure_otherFailure (2),18,2473,592,1881,38,0,38,10360,23.87066,5.71429,18.15637,0.38903,0.0,0.44817
7,SN_HO,MCG_Failure_otherFailure (2),54,155,153,2,485,224,261,21150,0.73286,0.7234,0.00946,2.30985,1.06682,1.23416
8,MCG_Failure_otherFailure (2),SN_HO,54,4104,153,3951,1006,82,924,21150,19.40426,0.7234,18.68085,4.79116,0.39053,5.37241
9,LTE_HO,MCG_Failure_otherFailure (2),93,174,87,87,2602,231,2371,35269,0.49335,0.24668,0.24668,7.39583,0.65659,6.73924


Dual Radio Statistics: sort by DL PLR_mlss (main lost, sub safe), ELR_mess (main excl, sub safe)


,ho_type0_m,ho_type0_s,ho_count,dl_lost_m,dl_lost_ms,dl_lost_mlss,dl_excl_m,dl_excl_ms,dl_excl_mess,dl_total,dl_PLR_m,dl_PLR_ms,dl_PLR_mlss,dl_ELR_m,dl_ELR_ms,dl_ELR_mess
0,SN_Rel,MN_HO,1,56,0,56,0,0,0,56,100.0,0.0,100.0,0.0,0.0,0.0
1,MCG_Failure_otherFailure (2),SN_Rel,3,266,0,266,0,0,0,266,100.0,0.0,100.0,0.0,0.0,0.0
2,NAS_Recovery_otherFailure (2),stable,2,1161,0,1161,0,0,0,1203,96.50873,0.0,96.50873,0.0,0.0,0.0
3,MCG_Failure_handoverFailure (1),SN_Setup,2,249,0,249,0,0,0,340,73.23529,0.0,73.23529,0.0,0.0,0.0
4,NAS_Recovery_otherFailure (2),LTE_HO,1,221,0,221,0,0,0,321,68.84735,0.0,68.84735,0.0,0.0,0.0
5,NAS_Recovery_otherFailure (2),SN_HO,1,144,0,144,0,0,0,244,59.01639,0.0,59.01639,0.0,0.0,0.0
6,SCG_Failure_t310-Expiry (0),SN_HO,1,215,0,215,2,0,2,369,58.26558,0.0,58.26558,0.54201,0.0,1.2987
7,MCG_Failure_handoverFailure (1),MN_HO,7,1343,0,1343,0,0,0,2725,49.2844,0.0,49.2844,0.0,0.0,0.0
8,MCG_Failure_handoverFailure (1),stable,11,4042,0,4042,23,0,23,10721,37.70171,0.0,37.70171,0.21453,0.0,0.34436
9,MCG_Failure_handoverFailure (1),MCG_Failure_otherFailure (2),2,1040,439,601,23,23,0,1619,64.23718,27.1155,37.12168,1.94915,1.94915,0.0
